In [1]:
import os
import sys
import datetime

#图像读取库
from PIL import Image
#矩阵运算库
import numpy as np
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(keras.__version__, keras.__path__)

2.3.0
2.4.0 ['/home/wilson/.local/lib/python3.8/site-packages/tensorflow/keras']


In [2]:
# 模型保存位置
model_save_path = 'emotion.model'

# 加载模型
model = None
if not os.path.exists(model_save_path):
    print("failed to find saved model in:", model_save_path)
    sys.exit(1)
else:
    model = keras.models.load_model(model_save_path)
    
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_192 (Functi (None, 6, 6, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1280)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 2,230,277
Non-trainable params: 34,112
_________________________________________________________________
